In [4]:
# 1. Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# 2. Load dataset
data = pd.read_csv("../Datasets/dengue.csv")

# 3. Encode categorical column 'Sex'
print(data['Sex'].unique())
data['Sex'] = data['Sex'].map({'Male': 0, 'Female': 1, 'Child': 2})
print(data['Sex'].isna().sum())
print(data[['Sex']].head())

# 4. Handle missing values
print('Missing values:', data.isnull().sum())
data = data.fillna(data.mean(numeric_only=True))

# 5. Features and Target split
X = data.drop('target', axis=1)
y = data['target'].astype(int)  # Ensure numeric labels
print(y.unique())
print(y.dtype)

# 6. Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 7. Define parameter grid for hyperparameter tuning
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'criterion': ['gini', 'entropy']
}

# 8. Initialize base model
rf = RandomForestClassifier(random_state=42)

# 9. Grid Search with Cross-Validation
grid_search = GridSearchCV(estimator=rf,
                           param_grid=param_grid,
                           cv=5,              # 5-fold cross-validation
                           n_jobs=-1,         # Use all CPU cores
                           verbose=1,
                           scoring='accuracy')

# 10. Fit Grid Search
grid_search.fit(X_train, y_train)

# 11. Best model from Grid Search
print("\nBest Parameters:", grid_search.best_params_)
best_model = grid_search.best_estimator_

# 12. Make predictions
y_pred = best_model.predict(X_test)

# 13. Evaluation
print('\nAccuracy:', accuracy_score(y_test, y_pred))
print('\nClassification Report:\n', classification_report(y_test, y_pred))
print('\nConfusion Matrix:\n', confusion_matrix(y_test, y_pred))


['Male' 'Female' 'Child']
0
   Sex
0    0
1    0
2    1
3    1
4    1
Missing values: Age                    0
Sex                    0
Haemoglobin            0
WBC Count             24
Differential Count     0
RBC PANEL              0
Platelet Count        17
PDW                   19
target                14
dtype: int64
[1 0]
int64
Fitting 5 folds for each of 72 candidates, totalling 360 fits

Best Parameters: {'criterion': 'gini', 'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 100}

Accuracy: 0.9900332225913622

Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.98      0.98        97
           1       0.99      1.00      0.99       204

    accuracy                           0.99       301
   macro avg       0.99      0.99      0.99       301
weighted avg       0.99      0.99      0.99       301


Confusion Matrix:
 [[ 95   2]
 [  1 203]]
